<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

-----


# Keras: Deep Learning library for TensorFlow



>Keras is a minimalist, highly modular neural networks library, written in Python and capable of running on top of either TensorFlow or Theano. 

>It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.
ref: https://keras.io/

<h2><a>Install instructions</a></h2>

Keras can be installed using `pip`:

** pip install keras **


<h2><a>Kaggle Challenge Data</a></h2>


>The Otto Group is one of the world’s biggest e-commerce companies, A consistent analysis of the performance of products is crucial. However, due to diverse global infrastructure, many identical products get classified differently.
For this competition, we have provided a dataset with 93 features for more than 200,000 products. The objective is to build a predictive model which is able to distinguish between our main product categories. 
Each row corresponds to a single product. There are a total of 93 numerical features, which represent counts of different events. All features have been obfuscated and will not be defined any further.

https://www.kaggle.com/c/otto-group-product-classification-challenge/data

##### For this section we will use the Kaggle Otto Group Challenge Data. You will find these data in 
`assets/data/kaggle_ottogroup/` folder.


<h2><a>Data Preparation</a></h2>

In [1]:
from scripts.kaggle_data import load_data, preprocess_data, preprocess_labels
import numpy as np
import matplotlib.pyplot as plt



import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)

/Users/halloran/anaconda/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train, labels = load_data('assets/data/kaggle_ottogroup/train.csv', train=True)
X_train, scaler = preprocess_data(X_train)
Y_train, encoder = preprocess_labels(labels)

X_test, ids = load_data('assets/data/kaggle_ottogroup/test.csv', train=False)
X_test, _ = preprocess_data(X_test, scaler)

nb_classes = Y_train.shape[1]
print(nb_classes, 'classes')

dims = X_train.shape[1]
print(dims, 'dims')

9 classes
93 dims


In [3]:
np.unique(labels)

array(['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'], dtype=object)

In [4]:
Y_train  # one-hot encoding

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)


<h2><a>Using Keras</a></h2>


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [6]:
dims = X_train.shape[1]
print(dims, 'dims')
print("Building model...")

nb_classes = Y_train.shape[1]
print(nb_classes, 'classes')

model = Sequential()
model.add(Dense(nb_classes, input_shape=(dims,), activation='sigmoid'))
model.add(Activation('softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(X_train, Y_train)

93 dims
Building model...
9 classes
Epoch 1/1
61878/61878 [==============================] - 2s 36us/step - loss: 1.9934


Simplicity is pretty impressive right? :)

Now lets understand:
<pre>The core data structure of Keras is a <b>model</b>, a way to organize layers. 

The main type of model is the <b>Sequential</b> model, a linear stack of layers.</pre>


What we did here is stacking a Fully Connected (<b>Dense</b>) layer of trainable weights from the input to the output and an <b>Activation</b> layer on top of the weights layer.


<h2><a>(some) others `keras.core.layers`</a></h2>


* `keras.layers.core.Flatten()`
* `keras.layers.core.Reshape(target_shape)`
* `keras.layers.core.Permute(dims)`

```python
model = Sequential()
model.add(Permute((2, 1), input_shape=(10, 64)))
# now: model.output_shape == (None, 64, 10)
# note: `None` is the batch dimension
```

* `keras.layers.core.Lambda(function, output_shape=None, arguments=None)`
* `keras.layers.core.ActivityRegularization(l1=0.0, l2=0.0)`

##### Activation

```python
from keras.layers.core import Activation

Activation(activation)
```

**Supported Activations** : [https://keras.io/activations/]

**Advanced Activations**: [https://keras.io/layers/advanced-activations/]

##### Optimizer

If you need to, you can further configure your optimizer. 

A core principle of Keras is to make things reasonably simple, while allowing the user to be fully in control when they need to (the ultimate control being the easy extensibility of the source code).

Here we used <b>SGD</b> (stochastic gradient descent) as an optimization algorithm for our trainable weights.  

<img src="http://sebastianruder.com/content/images/2016/09/saddle_point_evaluation_optimizers.gif" width="40%">

Source & Reference: http://sebastianruder.com/content/images/2016/09/saddle_point_evaluation_optimizers.gif


<h2><a>"Data Sciencing" this example a little bit more</a></h2>


What we did here is nice, however in the real world it is not useable because of overfitting.
Lets try and solve it with cross validation.

##### Overfitting


<img src="assets/images/overfitting.png">

To avoid overfitting, we will first split out data to training set and test set and test out model on the test set.
Next: we will use two of keras's callbacks <b>EarlyStopping</b> and <b>ModelCheckpoint</b>

Let's see first the model we implemented

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 9)                 846       
_________________________________________________________________
activation_1 (Activation)    (None, 9)                 0         
Total params: 846
Trainable params: 846
Non-trainable params: 0
_________________________________________________________________


In [8]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15, random_state=42)

fBestModel = 'assets/data/saved_models/best_model.h5' 
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1) 
best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)

model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs=50, 
          batch_size=128, verbose=True, callbacks=[best_model, early_stop]) 

Train on 52596 samples, validate on 9282 samples
Epoch 1/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.8672 - val_loss: 1.8577
Epoch 2/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.8477 - val_loss: 1.8401
Epoch 3/50
52596/52596 [==============================] - 1s 13us/step - loss: 1.8315 - val_loss: 1.8254
Epoch 4/50
52596/52596 [==============================] - 1s 11us/step - loss: 1.8177 - val_loss: 1.8127
Epoch 5/50
52596/52596 [==============================] - 1s 11us/step - loss: 1.8057 - val_loss: 1.8016
Epoch 6/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.7951 - val_loss: 1.7916
Epoch 7/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.7855 - val_loss: 1.7827
Epoch 8/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.7769 - val_loss: 1.7745
Epoch 9/50
52596/52596 [==============================] - 1s 12us/step - loss: 1.7689 - val_loss: 1.7670
Epoch 

<h2><a>Multi-Layer Fully Connected Networks</a></h2>


<img src="assets/images/MLP.png" width="45%">

#### Forward and Backward Propagation

<img src="assets/images/backprop.png" width="50%">

**Q:** _How hard can it be to build a Multi-Layer Fully-Connected Network with keras?_

**A:** _It is basically the same, just add more layers!_

In [10]:
model = Sequential()
model.add(Dense(100, input_shape=(dims,)))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               9400      
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 909       
_________________________________________________________________
activation_2 (Activation)    (None, 9)                 0         
Total params: 10,309
Trainable params: 10,309
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs=20, 
          batch_size=128, verbose=True)

Train on 52596 samples, validate on 9282 samples
Epoch 1/20
52596/52596 [==============================] - 1s 16us/step - loss: 1.1811 - val_loss: 0.8870
Epoch 2/20
52596/52596 [==============================] - 1s 13us/step - loss: 0.8101 - val_loss: 0.7944
Epoch 3/20
52596/52596 [==============================] - 1s 14us/step - loss: 0.7526 - val_loss: 0.7582
Epoch 4/20
52596/52596 [==============================] - 1s 14us/step - loss: 0.7245 - val_loss: 0.7373
Epoch 5/20
52596/52596 [==============================] - 1s 14us/step - loss: 0.7072 - val_loss: 0.7227
Epoch 6/20
52596/52596 [==============================] - 1s 13us/step - loss: 0.6948 - val_loss: 0.7120
Epoch 7/20
52596/52596 [==============================] - 1s 13us/step - loss: 0.6856 - val_loss: 0.7054
Epoch 8/20
52596/52596 [==============================] - 1s 13us/step - loss: 0.6787 - val_loss: 0.6999
Epoch 9/20
52596/52596 [==============================] - 1s 13us/step - loss: 0.6732 - val_loss: 0.6939
Epoch 